<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E6%94%BF%E5%BA%9C%E5%9C%B0%E7%90%86%E8%B3%87%E8%A8%8A%E6%92%88%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 抓周圍API

In [ ]:
import re
import time
import os
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from string import Template
import urllib
import json
import pickle

## 載入分行點位資料

In [ ]:
import pickle
with open('/content/drive/MyDrive/03_RESOURCE/GIS/brn_poi.pickle', 'rb') as f:
  poi = pickle.load(f)

In [ ]:
brn_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/taishin_brn.csv').drop_duplicates()
brn_xy_df = pd.read_csv('/content/drive/MyDrive/03_RESOURCE/GIS/brn_xy_df.csv')
brn_df = brn_df.merge(brn_xy_df,how='left' ,on = 'addr').copy()
brn_df = brn_df.drop_duplicates()

## API 設定模板

In [ ]:
imp_poi_tpl = Template("https://egis.moea.gov.tw/MoeaEGFxData_WebAPI_Inside/InnoServe/LandMark?resptype=GeoJson&x=$x&y=$y&buffer=$dist")
factory_tpl = Template("https://egis.moea.gov.tw/MoeaEGFxData_WebAPI_Inside/InnoServe/Factory?resptype=GeoJson&x=$x&y=$y&buffer=$dist")
bu_tpl = Template("https://egis.moea.gov.tw/MoeaEGFxData_WebAPI_Inside/InnoServe/BusinessBUSM?resptype=GeoJson&x=$x&y=$y&buffer=$dist")
cmp_tpl = Template("https://egis.moea.gov.tw/MoeaEGFxData_WebAPI_Inside/InnoServe/BusinessCMPY?resptype=GeoJson&x=$x&y=$y&buffer=$dist")
stk_tpl = Template("https://egis.moea.gov.tw/MoeaEGFxData_WebAPI_Inside/InnoServe/Stock?resptype=GeoJson&x=$x&y=$y&buffer=$dist")
code3_tpl = Template("https://egis.moea.gov.tw/MoeaEGFxData_WebAPI_Inside/InnoServe/CodeSearchEcon/GetInfo?resptype=GeoJson&x=$x&y=$y")

- imp_poi_tpl: 周圍重要地標
- factory_tpl: 周圍工廠 (工商登記資料)
- bu_tpl: 周圍商家 (工商登記資料)
- cmp_tpl: 周圍公司 (工商登記資料)
- stk_tpl: 周圍上市櫃公司 (工商登記資料)
- code3_tpl: 所屬經濟三級統計區代碼

[來源 經濟地理圖資中心](https://egis.moea.gov.tw/OpenData/)

In [ ]:
res = {}
failed_list = []

## For loop 爬取

In [ ]:

for qry in list(zip(brn_df['brn'], brn_df['lon'], brn_df['lat'])):
    print('獲取資訊', qry[0])
    api_sub_dict = {
        'x' : qry[1], 
        'y':qry[2], 
        'dist':500
    }
    imp_poi_url = imp_poi_tpl.substitute(api_sub_dict)
    factory_url = factory_tpl.substitute(api_sub_dict)
    bu_url= bu_tpl.substitute(api_sub_dict)
    cmp_url = cmp_tpl.substitute(api_sub_dict)
    stk_url = stk_tpl.substitute(api_sub_dict)
    code3_url = code3_tpl.substitute(api_sub_dict)

    try: 
        with urllib.request.urlopen(imp_poi_url) as url:
            imp_poi_data = json.loads(url.read().decode())

        with urllib.request.urlopen(factory_url) as url:
            factory_data = json.loads(url.read().decode())

        with urllib.request.urlopen(bu_url) as url:
            bu_data = json.loads(url.read().decode())

        with urllib.request.urlopen(cmp_url) as url:
            cmp_data = json.loads(url.read().decode())

        with urllib.request.urlopen(stk_url) as url:
            stk_data = json.loads(url.read().decode())

        with urllib.request.urlopen(code3_url) as url:
            code3_data = json.loads(url.read().decode())

        factory_num = len(factory_data['features'])
        bu_num = len(bu_data['features'])
        cmp_num = len(cmp_data['features'])
        stk_num = len(stk_data['features'])
        try:
            all_dict = {
              'poi_raw_data': [imp_poi_data, factory_data, bu_data, cmp_data, stk_data] ,
              'around_num': {'factory_num':factory_num, 'bu_num':bu_num, 'cmp_num':cmp_num, 'stk_num':stk_num},
              'code3_data': code3_data
            }
        except Exception as e:
            all_dict = {
              'poi_raw_data': [imp_poi_data, factory_data, bu_data, cmp_data, stk_data] ,
              'around_num': {'factory_num':factory_num, 'bu_num':bu_num, 'cmp_num':cmp_num, 'stk_num':stk_num},
              'code3_data': code3_data
            }

        print('成功')
        res[qry[0]] = all_dict
    except Exception as e:
        print(e)
        failed_list.append(qry[0])
        print('失敗')
    time.sleep(5)

In [ ]:
res = {}
failed_list = []

In [ ]:
## 失敗的重爬

#failed_df = brn_df[brn_df['brn'].isin([failed_list])]
failed_df = brn_df[brn_df['brn'].isin(['南崁分行'])]

for qry in list(zip(failed_df['brn'], failed_df['lon'], failed_df['lat'])):
    print('獲取資訊', qry[0])
    api_sub_dict = {
        'x' : qry[1], 
        'y':qry[2], 
        'dist':500
    }
    imp_poi_url = imp_poi_tpl.substitute(api_sub_dict)
    factory_url = factory_tpl.substitute(api_sub_dict)
    bu_url= bu_tpl.substitute(api_sub_dict)
    cmp_url = cmp_tpl.substitute(api_sub_dict)
    stk_url = stk_tpl.substitute(api_sub_dict)
    code3_url = code3_tpl.substitute(api_sub_dict)

    try:
        with urllib.request.urlopen(imp_poi_url) as url:
            imp_poi_data = json.loads(url.read().decode())

        with urllib.request.urlopen(factory_url) as url:
            factory_data = json.loads(url.read().decode())

        with urllib.request.urlopen(bu_url) as url:
            bu_data = json.loads(url.read().decode())

        with urllib.request.urlopen(cmp_url) as url:
            cmp_data = json.loads(url.read().decode())

        with urllib.request.urlopen(stk_url) as url:
            stk_data = json.loads(url.read().decode())


        with urllib.request.urlopen(code3_url) as url:
            code3_data = json.loads(url.read().decode())


        factory_num = len(factory_data['features'])
        bu_num = len(bu_data['features'])
        cmp_num = len(cmp_data['features'])
        stk_num = len(stk_data['features'])

        all_dict = {
          'poi_raw_data': [imp_poi_data, factory_data, bu_data, cmp_data, stk_data] ,
          'around_num': {'factory_num':factory_num, 'bu_num':bu_num, 'cmp_num':cmp_num, 'stk_num':stk_num},
          'code3_data': code3_data
        }
        print('成功')
        res[qry[0]] = all_dict

    except Exception as e:
        print(e)
        failed_list.append(qry[0])
        print('失敗')
    time.sleep(5)

獲取資訊 南崁分行
HTTP Error 500: Internal Server Error
失敗


南崁分行還是會無法爬到所屬經濟三級統計區，直接設定A6800005028

In [ ]:
'''
import pickle

with open ('/content/drive/MyDrive/03_RESOURCE/GIS/gis_open_api.pickle', 'wb') as f:
    pickle.dump(res, f)
'''

## 資料源整理

- 抓電子發票資料連結

https://egis.moea.gov.tw/EGIS_FILE/EGP_SD/107_B2C_SalesAmount.csv
https://egis.moea.gov.tw/EGIS_FILE/EGP_SD/107_B2C_StoreCount.csv
https://egis.moea.gov.tw/EGIS_FILE/EGP_SD/107_B2C_InvoiceCount.csv


- 二級發布區人口資料
- 二級發布區人口指標

https://data.gov.tw/dataset/18681
https://data.gov.tw/dataset/18624


- 村里綜合所得稅年收

https://data.gov.tw/dataset/103066


- 統計區比對服務: 給定地址就可以查詢所屬統計區，批次上傳最高筆數10000筆

https://moisagis.moi.gov.tw/moiap/match/system_common.cfm

經濟三級發布區之代碼應與二級發布區代碼需有對應關係存在，取得二級發布區之後兩碼作為擴充碼
- 經濟三級發布區: A6300003042
- 統計區二級發布區: A6303-42	

經濟三級發布區和二級發布區的劃分方式相同，但編碼格式不同

In [ ]:
with open('/content/drive/MyDrive/03_RESOURCE/GIS/gis_open_api.pickle', 'rb') as f:
    api_data = pickle.load(f)

# 抓人口統計資料

In [ ]:
%%capture


!mkdir pop_data
city_list = [
    '連江縣', '金門縣', '宜蘭縣', '新竹縣', '苗栗縣', '彰化縣', '南投縣', '雲林縣', '嘉義縣',
    '屏東縣', '臺東縣', '花蓮縣', '澎湖縣', '基隆市', '新竹市', '嘉義市', '臺北市', '高雄市',
    '新北市', '臺中市', '臺南市', '桃園市'
]


for city in city_list:
    city_zip = "'/content/pop_data/{}.zip'".format(city)
    pop_url = "'https://segis.moi.gov.tw/STAT/Generic/Project/GEN_STAT.ashx?method=downloadproductfile&code=3B0651D2CB3FE4C9933AB5B1C03BD952&STTIME=109Y12M&STUNIT=U0202&BOUNDARY={}'".format(city)
    ! wget {pop_url} -O {city_zip}
    !unzip {city_zip} -d /content/pop_data

files = Path('/content/pop_data').glob('**/*.csv')
csv_list = [str(x) for x in files]
csv_list.sort()
city_list.sort()

df_list = []
for file, city in zip(csv_list,city_list):
   df = pd.read_csv(file, encoding = 'CP950')
   df = df.iloc[1:,:].copy()
   df.insert(loc=0, column='city', value=city)
   df_list.append(df)
pd.concat(df_list,axis=0).to_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口資料', index=False)

In [ ]:
pd.concat(df_list,axis=0).to_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口資料.csv', index=False)

In [ ]:
%%capture


!mkdir pop_ratio
city_list = [
    '連江縣', '金門縣', '宜蘭縣', '新竹縣', '苗栗縣', '彰化縣', '南投縣', '雲林縣', '嘉義縣',
    '屏東縣', '臺東縣', '花蓮縣', '澎湖縣', '基隆市', '新竹市', '嘉義市', '臺北市', '高雄市',
    '新北市', '臺中市', '臺南市', '桃園市'
]


for city in city_list:
    city_zip = "'/content/pop_ratio/{}.zip'".format(city)
    pop_url = "'https://segis.moi.gov.tw/STAT/Generic/Project/GEN_STAT.ashx?method=downloadproductfile&code=D71B7495F95BC7C4184F1603ADC8AF64&STTIME=109Y12M&STUNIT=U0202&BOUNDARY={}'".format(city)
    ! wget {pop_url} -O {city_zip}
    !unzip {city_zip} -d /content/pop_ratio

files = Path('/content/pop_ratio').glob('**/*.csv')
csv_list = [str(x) for x in files]
csv_list.sort()
city_list.sort()

df_list = []
for file, city in zip(csv_list,city_list):
   df = pd.read_csv(file, encoding = 'CP950')
   df = df.iloc[1:,:].copy()
   df.insert(loc=0, column='city', value=city)
   df_list.append(df)
pd.concat(df_list,axis=0).to_csv('/content/drive/MyDrive/03_RESOURCE/GIS/台灣二級統計區人口指標.csv', index=False)